In [11]:
# setup
import pandas as pd
import numpy as np
from CalcBenchHandler import CalcBenchHandler as CBH

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# pull list of relevant calcbench companies
dfcorps = pd.read_csv('data/ciks/calcbench_companies.csv')
print(dfcorps.info())
dfcorps

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3581 entries, 0 to 3580
Data columns (total 9 columns):
ticker                     3581 non-null object
entity_name                3581 non-null object
entity_id                  3581 non-null int64
entity_code                3581 non-null int64
sic_code                   3581 non-null int64
naics                      3581 non-null int64
cik_code                   3581 non-null int64
naics_code                 3581 non-null int64
SICGroupMinorGroupTitle    3581 non-null object
dtypes: int64(6), object(3)
memory usage: 251.9+ KB
None


,ticker,entity_name,entity_id,entity_code,sic_code,naics,cik_code,naics_code,SICGroupMinorGroupTitle
0,RETC,12 Retech Corp,10621,1627611,7371,541511,1627611,541511,SERVICES-COMPUTER PROGRAMMING SERVICES
1,OPMZ,1PM Industries,10797,859747,2000,111998,859747,111998,FOOD AND KINDRED PRODUCTS
2,VNET,"21Vianet Group, Inc.",8423,1508475,7370,518210,1508475,518210,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING..."
3,XXII,"22nd Century Group, Inc.",2652,1347858,2111,312221,1347858,312221,CIGARETTES
4,TWOU,"2U, Inc.",9987,1459417,7372,334611,1459417,334611,SERVICES-PREPACKAGED SOFTWARE
...,...,...,...,...,...,...,...,...,...
3576,ZOOM,Zoom Technologies Inc,5989,822708,3661,999990,822708,999990,TELEPHONE & TELEGRAPH APPARATUS
3577,ZMTP,"Zoom Telephonics, Inc.",3977,1467761,3661,334210,1467761,334210,TELEPHONE & TELEGRAPH APPARATUS
3578,BPI,Zovio Inc,2284,1305323,8200,611430,1305323,611430,SERVICES-EDUCATIONAL SERVICES
3579,ZNGA,Zynga Inc,8550,1439404,7374,518210,1439404,518210,SERVICES-COMPUTER PROCESSING & DATA PREPARATION


In [6]:
from os import listdir
from os.path import isfile, join

# get list of tickers with available financials
ticker_dir = 'data/financials'
tickers = [f.split('.csv')[0] for f in listdir(ticker_dir) if isfile(join(ticker_dir, f))]

1171


['A', 'AA', 'AAN', 'AAP', 'AAPL']

In [109]:
# pull financials for test ticker
ticker = tickers[0]

ticker_path = join(ticker_dir, f'{ticker}.csv')

dffin = pd.read_csv(ticker_path)

# clean earnings_release_date
dffin.earnings_release_date = pd.to_datetime(dffin.earnings_release_date, errors='coerce')
dffin = dffin.dropna(subset=['earnings_release_date'])

# fill na income statement fields with with 0
cbh = CBH()
fs_cols = cbh.INS+cbh.CFS+cbh.BS
dffin = dffin.fillna(value={k: 0 for k in fs_cols})

# exclude unit based items like shares outstanding
share_cols = [
    'sharesoutstandingendofperiod',
    'avgsharesoutstandingbasic',
    'avgdilutedsharesoutstanding',
    'stockrepurchasedduringperiodshares'
]
per_share_cols = [
    'commonstockdividendspershare'
]
ins_base_col = ['revenueadjusted']
ins_cols = [c for c in cbh.INS if c not in share_cols+per_share_cols+ins_base_col]
cfs_cols = [c for c in cbh.CFS if c not in share_cols+per_share_cols+ins_base_col]
bs_cols = [c for c in cbh.BS if c not in share_cols+per_share_cols]

ins_chg_cols = [f'{c}_yoy_chg' for c in cbh.INS]
cfs_chg_cols = [f'{c}_yoy_chg' for c in cbh.CFS]
bs_chg_cols = [f'{c}_yoy_chg' for c in cbh.BS]

# yoy chg
dffin[ins_chg_cols] = dffin[cbh.INS]-dffin[cbh.INS].shift(4)
dffin[cfs_chg_cols] = dffin[cbh.CFS]-dffin[cbh.CFS].shift(4)
dffin[bs_chg_cols] = dffin[cbh.BS]-dffin[cbh.BS].shift(4)

# divide ins and cfs line items by revenue
# divide bs by assets or debt
dffin[ins_cols] = dffin[ins_cols].div(dffin.revenueadjusted, axis=0)
dffin[cfs_cols] = dffin[cfs_cols].div(dffin.revenueadjusted, axis=0)

asset_cols = ['cash', 'restrictedcashandinvestmentscurrent', 'availableforsalesecurities',
             'shortterminvestments', 'longterminvestments', 'totalinvestments', 'currentassets',
             'ppe']
debt_cols = [c for c in cbh.BS not in asset_cols+share_cols]

# TBU: THIS NEEDS TO BE DONE AFTER CALCULATING YOY CHG AND EV METRICS
dffin[asset_cols] = dffin[asset_cols].div(dffin.assets, axis=0)
dffin[debt_cols] = dffin[debt_cols].div(dffin.totaldebt, axis=0)

dffin.revenueadjusted = 1.0
dffin.assets = 1.0
dffin.totaldebt = 1.0

# divide share unit line items by end of period shares outstanding
dffin[share_cols] = dffin[share_cols].div(dffin.sharesoutstandingendofperiod, axis=0)

# sort by earnings_release_date ascending
dffin = dffin.sort_values(by=['earnings_release_date'])

# check all fields have values
assert dffin.isna().sum().sum() == 0, f'{ticker} has na fields!'

dffin.head(10)

	save_dir: None
	verbose: True


,earnings_release_date,filing_date,period,period_start,period_end,revenueadjusted,grossprofit,sgaexpense,researchanddevelopment,operatingexpenses,...,paymentsofdividends,paymentsofdividendscommonstock,paymentsofdividendspreferredstock,paymentsofdividendsnoncontrollinginterest,financingcashflow,stockrepurchasedduringperiodshares,stockrepurchasedduringperiodvalue,paymentsforrepurchaseofcommonstock,incometaxespaid,interestpaidnet
1,2009-08-17,2009-10-05,2009Q2,2009-05-01,2009-07-31,1.0,0.509934,0.366131,0.144749,0.510880,...,0.0,0.0,0.0,0.0,0.024598,0.0,0.000000,0.000000,0.0,0.0
2,2009-11-13,2009-12-21,2009Q3,2009-07-31,2009-10-31,1.0,-1.380463,0.353899,0.128535,-1.441302,...,0.0,0.0,0.0,0.0,0.652099,0.0,0.000000,0.000000,0.0,0.0
3,2010-02-12,2010-03-10,2009Q4,2009-11-01,2010-01-31,1.0,0.544106,0.343776,0.122836,0.466612,...,0.0,0.0,0.0,0.0,0.002473,0.0,0.082440,0.082440,0.0,0.0
4,2010-05-17,2010-06-07,2010Q1,2010-02-01,2010-04-30,1.0,0.121164,0.320220,0.118017,0.000000,...,0.0,0.0,0.0,0.0,-0.046420,0.0,0.129819,0.129819,0.0,0.0
5,2010-08-16,2010-10-06,2010Q2,2010-05-01,2010-07-31,1.0,0.083092,0.329480,0.111272,0.000000,...,0.0,0.0,0.0,0.0,0.486994,0.0,0.067919,0.067919,0.0,0.0
6,2010-11-12,2010-12-20,2010Q3,2010-07-31,2010-10-31,1.0,0.128807,0.299492,0.100888,0.000000,...,0.0,0.0,0.0,0.0,-0.010787,0.0,0.032995,0.032995,0.0,0.0
7,2011-02-14,2011-03-09,2010Q4,2010-11-01,2011-01-31,1.0,0.138907,0.293614,0.104674,0.000000,...,0.0,0.0,0.0,0.0,-1.075708,0.0,0.177749,0.177749,0.0,0.0
8,2011-05-13,2011-06-07,2011Q1,2011-02-01,2011-04-30,1.0,0.158617,0.279666,0.098390,0.000000,...,0.0,0.0,0.0,0.0,0.040549,0.0,0.000000,0.000000,0.0,0.0
9,2011-08-15,2011-09-07,2011Q2,2011-05-01,2011-07-31,1.0,0.166174,0.265523,0.095801,0.000000,...,0.0,0.0,0.0,0.0,-0.057363,0.0,0.113542,0.113542,0.0,0.0
10,2011-11-15,2011-12-16,2011Q3,2011-07-31,2011-10-31,1.0,0.181134,0.257523,0.094329,0.000000,...,0.0,0.0,0.0,0.0,-0.017361,0.0,0.020255,0.020255,0.0,0.0


In [99]:
# load bond-equity links and bond trades
link_path = 'data/ciks/bonds_to_equities_link.csv'
bond_px_path = 'data/bonds/clean_bond_close_pxs.csv'

dflink = pd.read_csv(link_path)
dfpxs = pd.read_csv(bond_px_path)

In [113]:
# clean dfpxs trans_dt
dfpxs.trans_dt = pd.to_datetime(dfpxs.trans_dt, errors='coerce')
dfpxs = dfpxs.dropna(subset=['trans_dt'])

# drop duplicate symbol/equity_cusip records
dfdupes = dflink.groupby(['SYMBOL', 'EQUITY_CUSIP']).count()
sym_counts = dfdupes.index.get_level_values(0).value_counts()
sym_dupes = sym_counts[sym_counts>1]
dfdupes = dfdupes.reset_index()
ser_eqy_cusip = dfdupes[~dfdupes.SYMBOL.isin(sym_dupes.index.values)].EQUITY_CUSIP
dflink = dflink[dflink.EQUITY_CUSIP.isin(ser_eqy_cusip)]

In [114]:
# get links to bond cusips for ticker
df_tick_links = dflink[dflink.SYMBOL == ticker]

# get pxs for ticker bond cusips
dftxs = dfpxs[dfpxs.cusip_id.isin(df_tick_links.cusip_id)]
dftxs.head()

,trans_dt,trd_rpt_efctv_dt,mtrty_dt,cusip_id,bond_sym_id,company_symbol,issuer_nm,debt_type_cd,scrty_ds,cpn_rt,close_pr,close_yld
0,2012-02-06,2007-10-29,2017-11-01,00846UAC5,A.GC,A,AGILENT TECHNOLOGIES INC,S-NT,Senior Unsecured Note,6.5,120.115,2.686534
1,2012-02-09,2007-10-29,2017-11-01,00846UAC5,A.GC,A,AGILENT TECHNOLOGIES INC,S-NT,Senior Unsecured Note,6.5,120.102,2.681065
2,2012-02-10,2007-10-29,2017-11-01,00846UAC5,A.GC,A,AGILENT TECHNOLOGIES INC,S-NT,Senior Unsecured Note,6.5,120.258,2.652957
3,2012-02-13,2007-10-29,2017-11-01,00846UAC5,A.GC,A,AGILENT TECHNOLOGIES INC,S-NT,Senior Unsecured Note,6.5,120.148,2.670143
4,2012-02-13,2007-10-29,2017-11-01,00846UAC5,A.GC,A,AGILENT TECHNOLOGIES INC,S-NT,Senior Unsecured Note,6.5,120.384,2.629966


In [128]:
# select last 2 years of financials for transaction
tx = dftxs.iloc[0]
df_2ltm = dffin[dffin.earnings_release_date <= tx.trans_dt].tail(8)
df_2ltm = df_2ltm.sort_values(by='earnings_release_date', ascending=False)
df_2ltm.index = pd.RangeIndex(df_2ltm.shape[0])
df_2ltm

,earnings_release_date,filing_date,period,period_start,period_end,revenueadjusted,grossprofit,sgaexpense,researchanddevelopment,operatingexpenses,...,paymentsofdividends,paymentsofdividendscommonstock,paymentsofdividendspreferredstock,paymentsofdividendsnoncontrollinginterest,financingcashflow,stockrepurchasedduringperiodshares,stockrepurchasedduringperiodvalue,paymentsforrepurchaseofcommonstock,incometaxespaid,interestpaidnet
0,2011-11-15,2011-12-16,2011Q3,2011-07-31,2011-10-31,1.0,0.181134,0.257523,0.094329,0.000000,...,0.0,0.0,0.0,0.0,-0.017361,0.0,0.020255,0.020255,0.0,0.0
1,2011-08-15,2011-09-07,2011Q2,2011-05-01,2011-07-31,1.0,0.166174,0.265523,0.095801,0.000000,...,0.0,0.0,0.0,0.0,-0.057363,0.0,0.113542,0.113542,0.0,0.0
2,2011-05-13,2011-06-07,2011Q1,2011-02-01,2011-04-30,1.0,0.158617,0.279666,0.098390,0.000000,...,0.0,0.0,0.0,0.0,0.040549,0.0,0.000000,0.000000,0.0,0.0
3,2011-02-14,2011-03-09,2010Q4,2010-11-01,2011-01-31,1.0,0.138907,0.293614,0.104674,0.000000,...,0.0,0.0,0.0,0.0,-1.075708,0.0,0.177749,0.177749,0.0,0.0
4,2010-11-12,2010-12-20,2010Q3,2010-07-31,2010-10-31,1.0,0.128807,0.299492,0.100888,0.000000,...,0.0,0.0,0.0,0.0,-0.010787,0.0,0.032995,0.032995,0.0,0.0
5,2010-08-16,2010-10-06,2010Q2,2010-05-01,2010-07-31,1.0,0.083092,0.329480,0.111272,0.000000,...,0.0,0.0,0.0,0.0,0.486994,0.0,0.067919,0.067919,0.0,0.0
6,2010-05-17,2010-06-07,2010Q1,2010-02-01,2010-04-30,1.0,0.121164,0.320220,0.118017,0.000000,...,0.0,0.0,0.0,0.0,-0.046420,0.0,0.129819,0.129819,0.0,0.0
7,2010-02-12,2010-03-10,2009Q4,2009-11-01,2010-01-31,1.0,0.544106,0.343776,0.122836,0.466612,...,0.0,0.0,0.0,0.0,0.002473,0.0,0.082440,0.082440,0.0,0.0


In [145]:
# convert last 2 years to one row
dftest = df_2ltm.head(1).copy()
col_desc = ['earnings_release_date', 'filing_date', 'period', 'period_start', 'period_end']
base_col_names = [c for c in df_2ltm.columns.values if c not in col_desc]
for i, row in df_2ltm.iterrows():
    for key in row.keys():
        if key not in col_desc:
            dftest[f'{key}_{i}'] = row[key]
dftest = dftest.drop(base_col_names, axis=1)
dftest

,earnings_release_date,filing_date,period,period_start,period_end,revenueadjusted_0,grossprofit_0,sgaexpense_0,researchanddevelopment_0,operatingexpenses_0,...,paymentsofdividends_7,paymentsofdividendscommonstock_7,paymentsofdividendspreferredstock_7,paymentsofdividendsnoncontrollinginterest_7,financingcashflow_7,stockrepurchasedduringperiodshares_7,stockrepurchasedduringperiodvalue_7,paymentsforrepurchaseofcommonstock_7,incometaxespaid_7,interestpaidnet_7
0,2011-11-15,2011-12-16,2011Q3,2011-07-31,2011-10-31,1.0,0.181134,0.257523,0.094329,0.0,...,0.0,0.0,0.0,0.0,0.002473,0.0,0.08244,0.08244,0.0,0.0
